Combined Recommendation Engine

In [8]:
import pandas as pd
import pickle as pkl

In [9]:
df_subset = pd.read_csv('recipe_w_images.csv')
apriori_rules = pd.read_csv('recipe_mix.csv')

In [10]:
import pickle as pkl

# Store cosine similarities pickle
with open('./df_subset.pkl', 'wb') as f:
    pkl.dump(df_subset, f)

In [11]:
df_subset[df_subset['name'] == 'moist gingerbread cake']

,Unnamed: 0,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,Image
2,2,moist gingerbread cake,537543,55,2001201872,2018-11-16,"['60-minutes-or-less', 'time-to-make', 'course...","[1617.0, 104.0, 213.0, 8.0, 40.0, 203.0, 80.0]",8,"['preheat the oven to 350&deg', 'f grease a lo...",a slightly sticky loaf cake flavoured with gin...,"['unsalted butter', 'applesauce', 'egg', 'unsu...",10,True


In [37]:
#dummy data
user_rated_recipes = {5856:2, 5991:4, 2004:5}

In [66]:
recipe_rules = list(apriori_rules['Base Product'].unique())
print(recipe_rules)

[486261, 495124, 495275, 495577, 494631, 514965, 494435, 487073, 494972, 494609, 494784, 495271, 494671, 494303, 495152, 487492, 487593, 495967, 487551, 496573, 495134, 487039, 514423, 487669, 493958, 518145, 518069, 496591, 475041, 496730, 518068, 518151, 493963, 496552, 497295, 497130, 497382, 497236, 497261, 518143, 487568, 494953, 475780, 475819, 518229, 496767]


In [38]:
def recommendation_engine(user_rated_recipes):
    for k, v in user_rated_recipes.items(): 
        if not v < 3:
            print(k, '>', v)

In [39]:
recommendation_engine(user_rated_recipes)

5991 > 4
2004 > 5


In [12]:
with open('cosine-similarities.pkl', 'rb') as f:

    cosine_sim3 = pkl.load(f) # deserialize using load()

In [13]:
# list of all recipe ids
ids = list(df_subset['id'])

# list to be used for column headers for cosine similarity dataframe
similarity_df = ['index'] + ids

In [14]:
# create cosine similarity pickle file

ids = list(df_subset['id'])

similarity_df = ['index'] + ids


cosine_similarity_df = pd.DataFrame(columns = similarity_df)
cosine_similarity_df

for i in range (1,len(ids)):
    
    # Populate info for new row
    row = [similarity_df[i]] + list(cosine_sim3[i-1])

    # Insert the new row into the DataFrame
    cosine_similarity_df.loc[len(cosine_similarity_df)] = row
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [15]:
cosine_similarity_df.head()

,index,537716,537671,537543,537485,537459,537458,537429,537351,537331,...,475022,475027,475062,475081,475109,475074,475060,475026,475029,475041
0,537716.0,1.000000,0.003346,0.000000,0.004317,0.017996,0.121125,0.036455,0.064252,0.103472,...,0.007395,0.038013,0.069278,0.024070,0.054522,0.060355,0.152348,0.0,0.019668,0.066139
1,537671.0,0.003346,1.000000,0.028755,0.000000,0.000000,0.000000,0.053193,0.013064,0.002875,...,0.015351,0.000000,0.017510,0.011017,0.020180,0.000000,0.004814,0.0,0.032017,0.000000
2,537543.0,0.000000,0.028755,1.000000,0.034434,0.000000,0.021920,0.052016,0.000000,0.069812,...,0.124735,0.045628,0.025461,0.021598,0.178912,0.000000,0.071526,0.0,0.000000,0.011613
3,537485.0,0.004317,0.000000,0.034434,1.000000,0.000000,0.004803,0.035178,0.000000,0.040126,...,0.094633,0.026880,0.000000,0.000000,0.005194,0.000000,0.012756,0.0,0.083408,0.004404
4,537459.0,0.017996,0.000000,0.000000,0.000000,1.000000,0.060057,0.000000,0.031128,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.018355


In [17]:
for i in ids:
    print(i)

537716
537671
537543
537485
537459
537458
537429
537351
537331
537323
537319
537241
537242
537175
537089
537076
537071
537073
537039
537037
536990
536977
536973
536930
536859
536836
536834
536843
536845
536747
536734
536726
536728
536729
536696
536700
536688
536679
536678
536662
536643
536610
536547
536575
536564
536568
536583
536603
536524
536501
536506
536474
536469
536465
536475
536476
536473
536464
536436
536446
536443
536455
536411
536401
536382
536384
536383
536363
536360
536345
536346
536322
536318
536305
536290
536288
536237
536226
536229
536221
536270
536212
536124
536119
536098
536066
536070
536086
536060
536063
536051
536044
536012
536013
536011
535958
535952
535907
535899
535874
535852
535853
535829
535831
535830
535781
535784
535782
535786
535783
535779
535724
535723
535722
535713
535673
535691
535684
535690
535651
535650
535653
535652
535644
535627
535602
535581
535570
535584
535571
535557
535555
535431
535389
535375
535371
535341
535337
535327
535321
535320
535318
535319

In [26]:
for i in range (1,len(ids)):
    
    # Populate info for new row
    row = [similarity_df[i]] + list(cosine_sim3[i-1])

    # Insert the new row into the DataFrame
    cosine_similarity_df.loc[len(cosine_similarity_df)] = row
    print(i)

,id,id_1,score_1,id_2,score_2,id_3,score_3,id_4,score_4,id_5,...,id_26,score_26,id_27,score_27,id_28,score_28,id_29,score_29,id_30,score_30


In [70]:
# create cosine similarity pickle file
top_30_cosine_similarity = ['id']
for i in range(1,31):
    top_30_cosine_similarity.append(f"id_{i}")
    top_30_cosine_similarity.append(f"score_{i}")

top_30_cosine_similarity_df = pd.DataFrame(columns = top_30_cosine_similarity)
# top_30_cosine_similarity_df.head()


#compile the top 30 recipes for each 
val = 0
for i in ids:
    print(val)
    val += 1
    cosine_similarity_df = cosine_similarity_df.sort_values(by=i, ascending=False)

    values = [i]

    
    for j in range(1,31):

        

        values.append(int(list(cosine_similarity_df['index'])[j]))
        values.append(list(cosine_similarity_df[i])[j])

    top_30_cosine_similarity_df.loc[len(top_30_cosine_similarity_df)] = values

    
top_30_cosine_similarity_df.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,id,id_1,score_1,id_2,score_2,id_3,score_3,id_4,score_4,id_5,...,id_26,score_26,id_27,score_27,id_28,score_28,id_29,score_29,id_30,score_30
0,537716.0,495705.0,0.331132,496709.0,0.301285,504083.0,0.300203,481997.0,0.293728,507069.0,...,502902.0,0.211165,496026.0,0.208722,489223.0,0.207061,481261.0,0.205652,498991.0,0.204586
1,537671.0,534266.0,0.525129,537089.0,0.446348,491099.0,0.380623,477064.0,0.378137,536506.0,...,483838.0,0.281753,488299.0,0.279957,502876.0,0.279513,488222.0,0.279091,528565.0,0.278193
2,537543.0,492410.0,0.526627,491474.0,0.425138,491511.0,0.413226,511697.0,0.400864,506726.0,...,509676.0,0.262070,518616.0,0.261922,511060.0,0.259648,482360.0,0.258343,532257.0,0.258129
3,537485.0,489383.0,0.389825,506922.0,0.343086,481124.0,0.340749,496040.0,0.336974,479412.0,...,488862.0,0.199942,483144.0,0.199496,504628.0,0.195071,500101.0,0.193232,511351.0,0.189071
4,537459.0,485835.0,0.655296,481351.0,0.360493,491868.0,0.328418,490045.0,0.312653,485911.0,...,492833.0,0.186912,517500.0,0.183236,521678.0,0.181751,476715.0,0.180648,482222.0,0.179698


In [71]:
top_30_cosine_similarity_df.shape

(9964, 61)

In [72]:
import pickle as pkl

# Store cosine similarities pickle
with open('./top_30_cosine_similarity.pkl', 'wb') as f:
    pkl.dump(top_30_cosine_similarity_df, f)

In [68]:
cosine_similarity_df = cosine_similarity_df.sort_values(by=537671, ascending=False)
cosine_similarity_df.head()

,index,537716,537671,537543,537485,537459,537458,537429,537351,537331,...,475022,475027,475062,475081,475109,475074,475060,475026,475029,475041
1,537671.0,0.003346,1.000000,0.028755,0.000000,0.000000,0.000000,0.053193,0.013064,0.002875,...,0.015351,0.000000,0.017510,0.011017,0.020180,0.000000,0.004814,0.000000,0.032017,0.000000
201,534266.0,0.039520,0.525129,0.000000,0.000000,0.051275,0.070779,0.020224,0.020849,0.000000,...,0.000000,0.000000,0.000000,0.017843,0.013081,0.000000,0.000000,0.000000,0.012224,0.012294
14,537089.0,0.020662,0.446348,0.094322,0.015562,0.000000,0.008188,0.021476,0.005695,0.073062,...,0.134532,0.086043,0.008719,0.013172,0.011555,0.021197,0.027277,0.000000,0.015970,0.020773
6067,491099.0,0.000000,0.380623,0.000000,0.008196,0.034273,0.000000,0.000000,0.042015,0.005953,...,0.014039,0.000000,0.046101,0.017716,0.000000,0.000000,0.009967,0.000000,0.012137,0.000000
9469,477064.0,0.000000,0.378137,0.000000,0.000000,0.000000,0.000000,0.000000,0.021768,0.000000,...,0.060848,0.000000,0.000000,0.000000,0.000000,0.023841,0.036216,0.158398,0.026153,0.000000


In [61]:
#compile the top 30 recipes for each 
for i in ids:
    cosine_similarity_df = cosine_similarity_df.sort_values(by=i, ascending=False)

    print(cosine_similarity_df.head())
    #[columns_to_keep]

    columns_to_keep = ['index', i]

    values = [i]
    
    for j in range(1,31):

        

        values.append(list(cosine_similarity_df['index'])[j])
        values.append(list(cosine_similarity_df[i])[j])

    print(values)

    

    break

         index    537716    537671    537543    537485    537459    537458  \
0     537716.0  1.000000  0.003346  0.000000  0.004317  0.017996  0.121125   
5156  495705.0  0.331132  0.010159  0.046691  0.003652  0.000000  0.075448   
4980  496709.0  0.301285  0.013900  0.000000  0.000000  0.000000  0.027713   
3297  504083.0  0.300203  0.000000  0.000000  0.000000  0.017581  0.092013   
8351  481997.0  0.293728  0.006311  0.000000  0.072141  0.000000  0.074636   

        537429    537351    537331  ...    475022    475027    475062  \
0     0.036455  0.064252  0.103472  ...  0.007395  0.038013  0.069278   
5156  0.045633  0.069393  0.118488  ...  0.006255  0.086913  0.069529   
4980  0.050448  0.007131  0.020378  ...  0.000000  0.008681  0.107345   
3297  0.018099  0.055487  0.199174  ...  0.000000  0.075387  0.032316   
8351  0.051288  0.012992  0.090776  ...  0.013948  0.055883  0.068470   

        475081    475109    475074    475060    475026    475029    475041  
0     0.024070 

In [119]:
def get_recommendations(recipe_index, cosine_sim=cosine_sim3):

    remaining = 30 - min(30,len(apriori_rules[apriori_rules['Base Product'] == recipe_index]))


    indices = df_subset[['id', 'name']]

    idx = indices[indices['id'] == recipe_index].index[0]
    
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 10 most similar movies
    top_similar = sim_scores[1:remaining+1]

    # Get the recipe indices and corresponding recipe similarity score
    recipe_indices = [i[0] for i in top_similar]
    recipe_similarity_scores = [i[1] for i in top_similar]

    recommendations_df = df_subset['name'].iloc[recipe_indices].to_frame().reset_index()
    recommendations_df['score'] = recipe_similarity_scores

# Return the top 10 most similar recipes
    return recommendations_df

In [120]:

get_recommendations(537543)  


,index,name,score
0,5856,gluten free gingerbread cake,0.526627
1,5991,gingerbread spice blend,0.425138
2,5983,tartine s soft glazed gingerbread,0.413226
3,2004,gingerbread smoothie,0.400864
4,2670,pumpkin gingerbread w caramel sauce,0.400677
5,4646,homemade brown sugar,0.336926
6,1281,ginger snaps,0.327445
7,5891,crunchy gingerbread cookies,0.324020
8,7079,applesauce apple butter,0.320176
9,7455,cardamom ginger crunch,0.317110


In [121]:
# pickle apriori rule csv
with open('./apriori-rules.pkl', 'wb') as f:
    pkl.dump(apriori_rules, f)

# add method to update apriori rules and resave csv + pickle